# **Koneksi Ke Google Big Query**

In [2]:
from google.colab import auth
from google.colab import data_table
from google.cloud import bigquery
from pandas_gbq import to_gbq
import pandas as pd
import numpy as np

# PRoses autentikasi akun
auth.authenticate_user()
print('Authenticated')

Authenticated


In [4]:
from google.cloud import bigquery

# Buat BigQuery client
project_id = 'skilful-sensor-468912-h9'
bq_client = bigquery.Client(project = project_id)

## **Proses Ekstraksi Data dari sumber (Extract)**

In [5]:
%%bigquery data_transaksi --project {project_id}

-- Tulis Query disini
SELECT * FROM dqlab-468906.dqcommerce.tbl_transaction

Query is running:   0%|          |

Downloading:   0%|          |

Setelah query dijalankan maka akan disimpan sebagai dataframe pandas

## **Periksa Informasi Umum pada Data**

In [6]:
#Periksa info umum pada data
data_transaksi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 829262 entries, 0 to 829261
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   trx_id      829257 non-null  object
 1   product_id  829262 non-null  object
 2   trx_date    829262 non-null  Int64 
 3   trx_time    829262 non-null  dbtime
 4   units       829251 non-null  Int64 
dtypes: Int64(2), dbtime(1), object(2)
memory usage: 33.2+ MB


## **Proses Transformasi Data (Transform)**

In [7]:
%%bigquery data_product --project {project_id}

-- Tulis Query disini
SELECT * FROM dqlab-468906.dqcommerce.tbl_product

Query is running:   0%|          |

Downloading:   0%|          |

In [12]:
data_product.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37 entries, 0 to 36
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   product_id        37 non-null     object
 1   product_name      37 non-null     object
 2   product_category  37 non-null     object
 3   product_cost      37 non-null     object
 4   product_price     37 non-null     object
dtypes: object(5)
memory usage: 1.6+ KB


In [8]:
%%bigquery data_funnel --project {project_id}

-- Tulis Query disini
SELECT * FROM dqlab-468906.dqcommerce.tbl_funnel

Query is running:   0%|          |

Downloading:   0%|          |

In [14]:
data_transaksi.sample(10)

,trx_id,product_id,trx_date,trx_time,units
279795,DQTrx181396,DQProduk-010,2024-06-17,13:56:12,1
91786,DQTrx572477,DQProduk-003,2025-04-13,18:28:12,1
127108,DQTrx110687,DQProduk-006,2024-04-18,18:48:47,1
444162,DQTrx464402,DQProduk-019,2025-02-01,09:16:49,1
580325,DQTrx236029,DQProduk-027,2024-08-04,15:22:20,1
581149,DQTrx257574,DQProduk-027,2024-08-25,16:17:05,1
454791,DQTrx638126,DQProduk-019,2025-05-24,12:24:13,1
362353,DQTrx716302,DQProduk-015,2025-07-12,20:46:37,1
9807,DQTrx114563,DQProduk-001,2024-04-22,07:12:22,1
1635,DQTrx018946,DQProduk-001,2024-01-21,22:37:33,1


In [10]:
data_transaksi['trx_date'] = data_transaksi['trx_date'].astype('str')

In [11]:
data_transaksi['trx_date'] = np.where(data_transaksi['trx_date'].str.len() == 7, '0' + data_transaksi['trx_date'], data_transaksi['trx_date'])
data_transaksi['trx_date'] = pd.to_datetime(data_transaksi['trx_date'], format = '%d%m%Y')
data_transaksi.sample(10)

,trx_id,product_id,trx_date,trx_time,units
26718,DQTrx390154,DQProduk-001,2024-12-13,11:44:02,1
256276,DQTrx118402,DQProduk-009,2024-04-26,02:39:31,1
362148,DQTrx709147,DQProduk-015,2025-07-07,11:12:50,1
696224,DQTrx486990,DQProduk-003,2025-02-17,13:07:28,2
550550,DQTrx549866,DQProduk-025,2025-03-30,15:45:07,1
545901,DQTrx456859,DQProduk-025,2025-01-27,04:40:34,1
363671,DQTrx772097,DQProduk-015,2025-08-19,11:33:15,1
331274,DQTrx142280,DQProduk-014,2024-05-15,22:31:31,1
167519,DQTrx770730,DQProduk-006,2025-08-18,11:06:04,1
815249,DQTrx714526,DQProduk-009,2025-07-11,15:10:18,4


In [15]:
data_transaksi['trx_id'].isna().sum()

np.int64(5)

In [17]:
data_transaksi.shape

(829262, 5)

In [19]:
# Hapus data yang kosong(null)
data_transaksi = data_transaksi[data_transaksi['trx_id'].notnull()]
data_transaksi.shape

(829257, 5)

In [26]:
data_transaksi['units'] = data_transaksi['units'].fillna(0)
data_transaksi[data_transaksi['units'].isna()]

,trx_id,product_id,trx_date,trx_time,units


In [31]:
data_transaksi[data_transaksi['units'] == 0]

,trx_id,product_id,trx_date,trx_time,units


In [34]:
for col in ['product_cost', 'product_price']:
  data_product[col] = data_product[col].str.replace('IDR', '')
  data_product[col] = data_product[col].astype(pd.Int64Dtype())

data_product.head()

,product_id,product_name,product_category,product_cost,product_price
0,DQProduk-009,Pulpen Metal DQLab,Alat Tulis,149850,164850
1,DQProduk-003,Sticky Notes DQLab,Alat Tulis,29850,59850
2,DQProduk-025,Tipe-X Roller DQLab,Alat Tulis,29850,44850
3,DQProduk-016,Set Pulpen DQLab,Alat Tulis,44850,149850
4,DQProduk-028,Penanda Warna DQLab,Alat Tulis,59850,164850


In [35]:
data_transaksi.duplicated(subset = 'trx_id').sum()

np.int64(0)

In [37]:
data_transaksi.duplicated().sum()

np.int64(0)

In [38]:
data_product.duplicated().sum()

np.int64(2)

In [40]:
data_product[data_product.duplicated(subset='product_id', keep = False)]

,product_id,product_name,product_category,product_cost,product_price
17,DQProduk-035,Kartu Ucapan DQLab,Kertas & Cetak,59850,119850
18,DQProduk-035,Kartu Ucapan DQLab,Kertas & Cetak,59850,119850
27,DQProduk-015,Mousepad Custom DQLab,Merchandise,59850,89850
28,DQProduk-015,Mousepad Custom DQLab,Merchandise,59850,89850


In [45]:
data_product = data_product.drop_duplicates(subset = ['product_id'], keep = 'first')
data_product.shape

(35, 5)

In [46]:
data_integration = data_transaksi.merge(data_product, on = 'product_id', how = 'left')
data_integration.head()

,trx_id,product_id,trx_date,trx_time,units,product_name,product_category,product_cost,product_price
0,DQTrx829214,DQProduk-016,2025-09-30,01:18:05,True,Set Pulpen DQLab,Alat Tulis,44850,149850
1,DQTrx829166,DQProduk-017,2025-09-30,18:58:33,True,Set Tulis Eksekutif DQLab,Merchandise,209850,299850
2,DQTrx829118,DQProduk-018,2025-09-30,06:58:09,True,Printer Mini DQLab,Elektronik,524850,599850
3,DQTrx829186,DQProduk-019,2025-09-30,09:28:13,True,Kertas Warna DQLab,Kertas & Cetak,209850,239850
4,DQTrx829212,DQProduk-019,2025-09-30,15:00:55,True,Kertas Warna DQLab,Kertas & Cetak,209850,239850


## **Proses Simpan Data (Load)**

In [57]:
import pandas as pd

# Upload ke BigQuery - tabel belum ada
to_gbq(
    data_integration,
    destination_table = 'tbl_dwh_dqcommerce.tbl_integration1',
    project_id = project_id,
    if_exists = 'fail'
)

100%|██████████| 1/1 [00:00<00:00, 10591.68it/s]


---

### Overview Total Penjualan Periode Data Tiap Bulan

In [50]:
data_transaksi['month_year'] = data_transaksi['trx_date'].dt.strftime('%Y-%m')
data_transaksi.head()

,trx_id,product_id,trx_date,trx_time,units,month_year
0,DQTrx829214,DQProduk-016,2025-09-30,01:18:05,True,2025-09
1,DQTrx829166,DQProduk-017,2025-09-30,18:58:33,True,2025-09
2,DQTrx829118,DQProduk-018,2025-09-30,06:58:09,True,2025-09
3,DQTrx829186,DQProduk-019,2025-09-30,09:28:13,True,2025-09
4,DQTrx829212,DQProduk-019,2025-09-30,15:00:55,True,2025-09


In [ ]:
penjualan_bulanan = data_transaksi.groupby(['month_year'], as_index = False).agg(total = ('units', 'sum'))
penjualan_bulanan.head()

,month_year,total
0,2024-01,38009
1,2024-02,36935
2,2024-03,39981
3,2024-04,47102
4,2024-05,46910


In [58]:
import plotly.express as px

# Membuat line chart
fig = px.line(
    penjualan_bulanan,
    x="month_year",
    y="total",
    title="<b>Grafik Total Penjualan Bulanan DQ-Commerce</b>",
    color_discrete_sequence = ['#39074E'],
    markers=False)

"""
fig.add_vrect(
    x0="2025-03", x1="2025-09",
    fillcolor="#E73c35",
    opacity=0.5,
    layer="below",
    line_width=0,
    #annotation_text="Tahun 2025",
    #annotation_position="top "
)
"""

fig.update_layout(
    height = 450,
    width = 1100,
    plot_bgcolor = 'rgba(0, 0, 0, 0)',
    yaxis = dict(
        showline = True,
        linewidth = 1,
        color = '#544018',
        linecolor = '#544018',
        showgrid=False,
    ),
    xaxis = dict(
        title = '',
        showline = True,
        linewidth = 1,
        color = '#544018',
        linecolor = '#544018',
        showgrid=False,
    ),
)

fig.show()

NameError: name 'penjualan_bulanan' is not defined

In [52]:
penjualan_per_produk = data_transaksi.groupby(['month_year', 'product_id'], as_index = False).agg(total = ('units', 'sum'))
penjualan_per_produk.head()

,month_year,product_id,total
0,2025-09,DQProduk-016,1
1,2025-09,DQProduk-017,1
2,2025-09,DQProduk-018,1
3,2025-09,DQProduk-019,4
4,2025-09,DQProduk-024,1


In [53]:
import plotly.express as px

# Membuat line chart
fig = px.line(
    penjualan_per_produk, #[penjualan['month_year'] >= '2025-01'],
    x="month_year",
    y="total",
    color="product_id",
    title="<b>Grafik Total Penjualan Bulanan DQ-Commerce Per Produk</b>",
    markers=True
)

fig.update_layout(
    height = 550,
    width = 1200,
    plot_bgcolor = 'rgba(0, 0, 0, 0)',
    yaxis = dict(
        showline = True,
        linewidth = 1,
        color = '#544018',
        linecolor = '#544018',
        showgrid=False,
    ),
    xaxis = dict(
        title = '',
        showline = True,
        linewidth = 1,
        color = '#544018',
        linecolor = '#544018',
        showgrid=False,
    ),
)

fig.show()

In [54]:
import plotly.graph_objects as go

highlight_product = ['DQProduk-003', 'DQProduk-019', 'DQProduk-025', 'DQProduk-018', 'DQProduk-031']

fig = go.Figure()

# Loop setiap produk
for product in penjualan_per_produk['product_id'].unique():
    df_product = penjualan_per_produk[penjualan_per_produk['product_id'] == product]

    # Tentukan properti styling
    if product in highlight_product:
        line_color = 'purple'
        line_width = 3
    else:
        line_color = 'lightgray'
        line_width = 1.2

    # Tambahkan trace
    fig.add_trace(go.Scatter(
        x=df_product['month_year'],
        y=df_product['total'],
        mode='lines+markers',
        name=product,
        line=dict(
            width=line_width,
            color=line_color
        )
    ))

# Layout styling
fig.update_layout(
    title="<b>Grafik Total Penjualan Bulanan DQ-Commerce Per Produk</b>",
    height=550,
    width=1200,
    plot_bgcolor='rgba(0, 0, 0, 0)',
    xaxis=dict(
        showline=True,
        linewidth=1,
        color='#544018',
        linecolor='#544018',
        title='',
        showgrid=False,
    ),
    yaxis=dict(
        showline=True,
        linewidth=1,
        color='#544018',
        linecolor='#544018',
        showgrid=False,
    )
)

fig.show()